# Momentum, Value, and Size Premiums
## Financial Technology - AI Trading Assignment

This notebook tests whether classic asset pricing factors (market beta, size, value, momentum) show up in a large sample of U.S. stocks using firm-level regressions on merged CRSP-Compustat monthly data.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.4f}'.format)

---
## Question 1: Load and Explore the Data
Load the data. Report the number of observations, the date range, and the number of unique firms.

In [ ]:
# DATA NOTE:
# Parquet file is stored locally and NOT tracked in GitHub
# Update DATA_PATH as needed

DATA_PATH = r"D:\AI Trading - Fin Tech\stock_data_filtered.parquet"
df = pd.read_parquet(DATA_PATH)

# Ensure date column is datetime
df['MthCalDt'] = pd.to_datetime(df['MthCalDt'])

print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"Number of observations:  {len(df):,}")
print(f"Number of columns:       {df.shape[1]}")
print(f"Number of unique firms:  {df['PERMNO'].nunique():,}")
print(f"Date range:              {df['MthCalDt'].min().strftime('%Y-%m')} to {df['MthCalDt'].max().strftime('%Y-%m')}")
print(f"Number of unique months: {df['MthCalDt'].nunique()}")
print("\nReturn (MthRet) summary statistics:")
print(df['MthRet'].describe())

---
## Question 2: Define the Variables

Construct the following factor characteristics for each firm-month:
- **(a) Market Beta** - rolling 60-month regression of firm returns on equal-weighted market return
- **(b) Size** - log of market capitalization (price x shares outstanding)
- **(c) Value (Book-to-Market)** - book equity / market equity
- **(d) Momentum** - cumulative return over months t-12 to t-2 (skip most recent month)

In [ ]:
# Sort data for rolling calculations
df = df.sort_values(['PERMNO', 'MthCalDt']).reset_index(drop=True)

# ------------------------------------------------------------------
# (b) SIZE: log of market capitalization
# Market cap = share price (prcc_c) * shares outstanding (csho)
# prcc_c is calendar-year price; csho is in millions
# ------------------------------------------------------------------
df['MktCap'] = df['prcc_c'].abs() * df['csho']  # abs() handles negative prices (bid-ask)
df['Size'] = np.log(df['MktCap'].replace(0, np.nan))

print("Size (log market cap) summary:")
print(df['Size'].describe())

In [ ]:
# ------------------------------------------------------------------
# (c) VALUE: Book-to-Market ratio
# Book equity = stockholders' equity (seq) + deferred taxes (txdb, 0 if missing)
#              - preferred stock (pstkl or pstk, 0 if missing)
# Market equity = MktCap
# ------------------------------------------------------------------
df['BookEquity'] = df['seq'].fillna(df['ceq'] + df['pstk'].fillna(0))
df['BookEquity'] = df['BookEquity'] + df['txdb'].fillna(0) - df['pstkl'].fillna(df['pstk'].fillna(0))

df['BM'] = df['BookEquity'] / df['MktCap'].replace(0, np.nan)

# Winsorize B/M at 1st and 99th percentiles to handle outliers
bm_lower = df['BM'].quantile(0.01)
bm_upper = df['BM'].quantile(0.99)
df['BM'] = df['BM'].clip(lower=bm_lower, upper=bm_upper)

print("Book-to-Market ratio summary:")
print(df['BM'].describe())

In [ ]:
# ------------------------------------------------------------------
# (a) MARKET BETA: rolling 60-month regression on equal-weighted market return
# First compute the equal-weighted market return each month
# ------------------------------------------------------------------
mkt_ret = df.groupby('MthCalDt')['MthRet'].mean().rename('MktRet')
df = df.merge(mkt_ret, on='MthCalDt', how='left')

def rolling_beta(group, window=60, min_periods=24):
    """Compute rolling beta for a single firm using vectorized rolling covariance."""
    ret = group['MthRet']
    mkt = group['MktRet']
    cov = ret.rolling(window=window, min_periods=min_periods).cov(mkt)
    var = mkt.rolling(window=window, min_periods=min_periods).var()
    return cov / var

print("Computing rolling betas (this may take a few minutes)...")
df['Beta'] = df.groupby('PERMNO', group_keys=False).apply(rolling_beta)

# Winsorize beta at 1st and 99th percentiles
beta_lower = df['Beta'].quantile(0.01)
beta_upper = df['Beta'].quantile(0.99)
df['Beta'] = df['Beta'].clip(lower=beta_lower, upper=beta_upper)

print("Market Beta summary:")
print(df['Beta'].describe())

In [ ]:
# ------------------------------------------------------------------
# (d) MOMENTUM: cumulative return from t-12 to t-2
# Skip the most recent month (t-1) to avoid short-term reversal
# ------------------------------------------------------------------
def compute_momentum(group):
    """Momentum = cumulative return from t-12 to t-2."""
    ret = group['MthRet']
    # Cumulative return over 11 months (t-12 to t-2): product of (1+r) then subtract 1
    cum_12 = (1 + ret).rolling(window=11, min_periods=6).apply(np.prod, raw=True) - 1
    # Shift by 1 to skip the most recent month (use returns from t-12 to t-2, not t-1)
    return cum_12.shift(1)

print("Computing momentum...")
df['Momentum'] = df.groupby('PERMNO', group_keys=False).apply(compute_momentum)

# Winsorize momentum at 1st and 99th percentiles
mom_lower = df['Momentum'].quantile(0.01)
mom_upper = df['Momentum'].quantile(0.99)
df['Momentum'] = df['Momentum'].clip(lower=mom_lower, upper=mom_upper)

print("Momentum summary:")
print(df['Momentum'].describe())

In [ ]:
# ------------------------------------------------------------------
# Create next-month return (the dependent variable)
# ------------------------------------------------------------------
df['NextRet'] = df.groupby('PERMNO')['MthRet'].shift(-1)

print("Next-month return summary:")
print(df['NextRet'].describe())

# Summary of factor coverage
factors = ['Beta', 'Size', 'BM', 'Momentum', 'NextRet']
print("\n" + "="*60)
print("FACTOR VARIABLE COVERAGE")
print("="*60)
for col in factors:
    n_valid = df[col].notna().sum()
    pct = 100 * n_valid / len(df)
    print(f"{col:12s}: {n_valid:>10,} non-missing ({pct:.1f}%)")

# Create regression-ready sample
reg_df = df.dropna(subset=factors).copy()
print(f"\nRegression sample size: {len(reg_df):,} observations")
print(f"Unique firms in sample: {reg_df['PERMNO'].nunique():,}")
print(f"Date range: {reg_df['MthCalDt'].min().strftime('%Y-%m')} to {reg_df['MthCalDt'].max().strftime('%Y-%m')}")

---
## Question 4: Single-Factor Model

Run a pooled OLS regression of next-month firm returns on market beta only, with standard errors clustered by firm and month (double-clustered) to correct for correlation in the error term.

In [ ]:
# ------------------------------------------------------------------
# Pooled OLS: NextRet ~ Beta
# Double-clustered standard errors (by firm and by month)
# ------------------------------------------------------------------

# Prepare data
X_beta = sm.add_constant(reg_df['Beta'])
y = reg_df['NextRet']

# Fit OLS
model_q4 = OLS(y, X_beta).fit(
    cov_type='cluster',
    cov_kwds={'groups': reg_df[['PERMNO', 'MthCalDt']].values}
)

print("="*60)
print("QUESTION 4: Single-Factor Model (Beta Only)")
print("Dependent variable: Next-month return")
print("Standard errors: Double-clustered by firm and month")
print("="*60)
print(model_q4.summary().tables[1])
print(f"\nN = {int(model_q4.nobs):,}")
print(f"R-squared = {model_q4.rsquared:.6f}")

---
## Question 5: Full Factor Model

Run the full model with all four factors: market beta, size, value, and momentum. Use pooled OLS with double-clustered standard errors.

**(a)** Report coefficient estimates and t-statistics.  
**(b)** Interpret the signs of the coefficients.  
**(c)** Compare with Jegadeesh & Titman (1993) and Fama & French (2015).

In [ ]:
# ------------------------------------------------------------------
# Pooled OLS: NextRet ~ Beta + Size + BM + Momentum
# Double-clustered standard errors (by firm and by month)
# ------------------------------------------------------------------

factor_cols = ['Beta', 'Size', 'BM', 'Momentum']
X_full = sm.add_constant(reg_df[factor_cols])
y = reg_df['NextRet']

model_q5 = OLS(y, X_full).fit(
    cov_type='cluster',
    cov_kwds={'groups': reg_df[['PERMNO', 'MthCalDt']].values}
)

print("="*60)
print("QUESTION 5: Full Factor Model")
print("Dependent variable: Next-month return")
print("Standard errors: Double-clustered by firm and month")
print("="*60)
print(model_q5.summary().tables[1])
print(f"\nN = {int(model_q5.nobs):,}")
print(f"R-squared = {model_q5.rsquared:.6f}")
print(f"Adj. R-squared = {model_q5.rsquared_adj:.6f}")

# Detailed results table
print("\n" + "="*60)
print("(a) Coefficient Estimates and T-Statistics")
print("="*60)
results_table = pd.DataFrame({
    'Coefficient': model_q5.params,
    'Std Error': model_q5.bse,
    't-statistic': model_q5.tvalues,
    'p-value': model_q5.pvalues
})
print(results_table.to_string(float_format='{:.6f}'.format))

print("\n" + "="*60)
print("(b) Interpretation of Coefficient Signs")
print("="*60)
for var in factor_cols:
    coef = model_q5.params[var]
    pval = model_q5.pvalues[var]
    sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
    direction = "positive" if coef > 0 else "negative"
    print(f"  {var:12s}: {coef:+.6f} ({direction}) {sig}")

print("\n(c) Consistency with literature:")
print("  - Jegadeesh & Titman (1993): Momentum should have a POSITIVE coefficient")
print("    (winners continue winning, losers continue losing)")
print("  - Fama & French (2015): Size should be NEGATIVE (small firms earn higher returns),")
print("    Value (B/M) should be POSITIVE (high B/M firms earn higher returns)")

---
## Question 6: Are the Effects Linear?

Add a quadratic term for each of the four factors. Test whether any are statistically significant.

In [ ]:
# ------------------------------------------------------------------
# Add quadratic terms: Beta^2, Size^2, BM^2, Momentum^2
# ------------------------------------------------------------------

for col in factor_cols:
    reg_df[col + '_sq'] = reg_df[col] ** 2

quad_cols = factor_cols + [c + '_sq' for c in factor_cols]
X_quad = sm.add_constant(reg_df[quad_cols])
y = reg_df['NextRet']

model_q6 = OLS(y, X_quad).fit(
    cov_type='cluster',
    cov_kwds={'groups': reg_df[['PERMNO', 'MthCalDt']].values}
)

print("="*60)
print("QUESTION 6: Model with Quadratic Terms")
print("Dependent variable: Next-month return")
print("Standard errors: Double-clustered by firm and month")
print("="*60)
print(model_q6.summary().tables[1])
print(f"\nN = {int(model_q6.nobs):,}")
print(f"R-squared = {model_q6.rsquared:.6f}")

# Highlight significant quadratic terms
print("\nQuadratic term significance:")
for col in factor_cols:
    sq_col = col + '_sq'
    coef = model_q6.params[sq_col]
    pval = model_q6.pvalues[sq_col]
    sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else "not significant"
    print(f"  {sq_col:15s}: coef={coef:+.6f}, p-value={pval:.4f}  [{sig}]")

---
## Question 7: Subsample Analysis

Re-run the full linear model on different subsamples to see if premiums have changed over time.

In [ ]:
# ------------------------------------------------------------------
# Split into roughly equal subperiods
# ------------------------------------------------------------------

reg_df['Year'] = reg_df['MthCalDt'].dt.year
min_year = reg_df['Year'].min()
max_year = reg_df['Year'].max()
mid_year = (min_year + max_year) // 2

# Define subperiods
third = (max_year - min_year) // 3
periods = {
    f'Early ({min_year}-{min_year + third})': (min_year, min_year + third),
    f'Middle ({min_year + third + 1}-{min_year + 2*third})': (min_year + third + 1, min_year + 2*third),
    f'Late ({min_year + 2*third + 1}-{max_year})': (min_year + 2*third + 1, max_year),
}

print("="*60)
print("QUESTION 7: Subsample Analysis")
print("="*60)

subsample_results = {}

for period_name, (start, end) in periods.items():
    sub = reg_df[(reg_df['Year'] >= start) & (reg_df['Year'] <= end)]
    if len(sub) < 100:
        continue
    
    X_sub = sm.add_constant(sub[factor_cols])
    y_sub = sub['NextRet']
    
    model_sub = OLS(y_sub, X_sub).fit(
        cov_type='cluster',
        cov_kwds={'groups': sub[['PERMNO', 'MthCalDt']].values}
    )
    
    subsample_results[period_name] = {
        'N': int(model_sub.nobs),
        'coefs': model_sub.params[factor_cols],
        'tvals': model_sub.tvalues[factor_cols],
        'pvals': model_sub.pvalues[factor_cols],
    }
    
    print(f"\n--- {period_name} (N={int(model_sub.nobs):,}) ---")
    for var in factor_cols:
        coef = model_sub.params[var]
        tval = model_sub.tvalues[var]
        pval = model_sub.pvalues[var]
        sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
        print(f"    {var:12s}: coef={coef:+.6f}, t={tval:+.3f} {sig}")

In [ ]:
# ------------------------------------------------------------------
# Visualize how factor premiums evolve over time
# ------------------------------------------------------------------

# Compare coefficients across subperiods
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Factor Premiums Across Subperiods', fontsize=14, fontweight='bold')

for idx, var in enumerate(factor_cols):
    ax = axes[idx // 2][idx % 2]
    period_names = list(subsample_results.keys())
    coefs = [subsample_results[p]['coefs'][var] for p in period_names]
    tvals = [subsample_results[p]['tvals'][var] for p in period_names]
    
    colors = ['green' if abs(t) > 1.96 else 'red' for t in tvals]
    bars = ax.bar(range(len(period_names)), coefs, color=colors, alpha=0.7, edgecolor='black')
    ax.set_title(f'{var} Premium', fontweight='bold')
    ax.set_ylabel('Coefficient')
    ax.set_xticks(range(len(period_names)))
    ax.set_xticklabels([p.split('(')[0].strip() for p in period_names], fontsize=9)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    # Add t-stat labels
    for i, (c, t) in enumerate(zip(coefs, tvals)):
        ax.text(i, c, f't={t:.2f}', ha='center', va='bottom' if c > 0 else 'top', fontsize=8)

plt.tight_layout()
plt.savefig('subsample_premiums.png', dpi=150, bbox_inches='tight')
plt.show()
print("Green = significant at 5%; Red = not significant")

---
## Question 8: Out-of-Sample Backtesting

Estimate the full linear model on a training set, predict returns on a test set, and form long-short portfolios. Report average return and Sharpe ratio.

In [ ]:
# ------------------------------------------------------------------
# Train/Test split: first 70% of months for training, last 30% for testing
# ------------------------------------------------------------------

unique_months = sorted(reg_df['MthCalDt'].unique())
split_idx = int(len(unique_months) * 0.7)
train_end = unique_months[split_idx]

train_df = reg_df[reg_df['MthCalDt'] <= train_end].copy()
test_df = reg_df[reg_df['MthCalDt'] > train_end].copy()

print(f"Training period: {train_df['MthCalDt'].min().strftime('%Y-%m')} to {train_df['MthCalDt'].max().strftime('%Y-%m')}")
print(f"Training observations: {len(train_df):,}")
print(f"\nTest period: {test_df['MthCalDt'].min().strftime('%Y-%m')} to {test_df['MthCalDt'].max().strftime('%Y-%m')}")
print(f"Test observations: {len(test_df):,}")

# Estimate model on training data
X_train = sm.add_constant(train_df[factor_cols])
y_train = train_df['NextRet']
model_train = OLS(y_train, X_train).fit()

print("\nTraining model coefficients:")
print(model_train.params)

In [ ]:
# ------------------------------------------------------------------
# Predict returns and form long-short portfolios in the test period
# ------------------------------------------------------------------

# Predict expected returns
X_test = sm.add_constant(test_df[factor_cols])
test_df['PredRet'] = model_train.predict(X_test)

# Each month: go long top quintile, short bottom quintile
portfolio_returns = []

for month, group in test_df.groupby('MthCalDt'):
    if len(group) < 20:
        continue
    
    # Rank stocks by predicted return
    group = group.copy()
    group['quintile'] = pd.qcut(group['PredRet'], 5, labels=False, duplicates='drop')
    
    # Long top quintile (4), short bottom quintile (0)
    long_ret = group[group['quintile'] == 4]['NextRet'].mean()
    short_ret = group[group['quintile'] == 0]['NextRet'].mean()
    ls_ret = long_ret - short_ret
    
    portfolio_returns.append({
        'Month': month,
        'Long': long_ret,
        'Short': short_ret,
        'LongShort': ls_ret,
        'N_stocks': len(group)
    })

port_df = pd.DataFrame(portfolio_returns)

print("="*60)
print("QUESTION 8: Out-of-Sample Backtesting Results")
print("="*60)

# Monthly statistics
avg_ls = port_df['LongShort'].mean()
std_ls = port_df['LongShort'].std()
sharpe_monthly = avg_ls / std_ls if std_ls > 0 else 0
sharpe_annual = sharpe_monthly * np.sqrt(12)

print(f"\nLong-Short Portfolio (monthly):")
print(f"  Average return:     {avg_ls:.4f} ({avg_ls*100:.2f}%)")
print(f"  Std deviation:      {std_ls:.4f}")
print(f"  Monthly Sharpe:     {sharpe_monthly:.4f}")
print(f"  Annualized Sharpe:  {sharpe_annual:.4f}")
print(f"  Number of months:   {len(port_df)}")

# T-test: is average return significantly different from zero?
t_stat = avg_ls / (std_ls / np.sqrt(len(port_df)))
print(f"  t-statistic:        {t_stat:.3f}")

print(f"\nLong portfolio avg return:   {port_df['Long'].mean():.4f} ({port_df['Long'].mean()*100:.2f}%)")
print(f"Short portfolio avg return:  {port_df['Short'].mean():.4f} ({port_df['Short'].mean()*100:.2f}%)")

In [ ]:
# ------------------------------------------------------------------
# Plot cumulative returns
# ------------------------------------------------------------------

port_df = port_df.sort_values('Month')
port_df['CumLong'] = (1 + port_df['Long']).cumprod()
port_df['CumShort'] = (1 + port_df['Short']).cumprod()
port_df['CumLS'] = (1 + port_df['LongShort']).cumprod()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Cumulative returns
ax1.plot(port_df['Month'], port_df['CumLS'], 'b-', linewidth=1.5, label='Long-Short')
ax1.plot(port_df['Month'], port_df['CumLong'], 'g--', linewidth=1, alpha=0.7, label='Long (Top Quintile)')
ax1.plot(port_df['Month'], port_df['CumShort'], 'r--', linewidth=1, alpha=0.7, label='Short (Bottom Quintile)')
ax1.axhline(y=1, color='black', linestyle='-', linewidth=0.5)
ax1.set_title('Cumulative Returns: Out-of-Sample Backtest', fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Cumulative Return ($1 invested)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Monthly L/S returns distribution
ax2.hist(port_df['LongShort'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
ax2.axvline(x=avg_ls, color='red', linestyle='--', linewidth=2, label=f'Mean = {avg_ls:.4f}')
ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_title('Distribution of Monthly Long-Short Returns', fontweight='bold')
ax2.set_xlabel('Monthly Return')
ax2.set_ylabel('Frequency')
ax2.legend()

plt.tight_layout()
plt.savefig('backtest_results.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Question 9: Factor Farming

Select 20 plausible accounting variables, normalize them cross-sectionally, test in-sample and out-of-sample.

In [ ]:
# ------------------------------------------------------------------
# (a) Select 20 plausible accounting variables as candidate factors
# ------------------------------------------------------------------

# These are economically motivated variables from the Compustat data:
candidate_factors = {
    'gp':       'Gross Profit (profitability)',
    'ebitda':   'EBITDA (operating profitability)',
    'ni':       'Net Income (bottom-line profitability)',
    'oibdp':    'Operating Income Before Depreciation',
    'sale':     'Total Sales (revenue)',
    'at':       'Total Assets (firm size alt.)',
    'capx':     'Capital Expenditures (investment)',
    'dltt':     'Long-Term Debt (leverage)',
    'che':      'Cash & Short-Term Investments (liquidity)',
    'invt':     'Total Inventory',
    'dp':       'Depreciation & Amortization',
    'xsga':     'SG&A Expense (overhead)',
    'xint':     'Interest Expense (debt burden)',
    'txt':      'Total Income Taxes',
    'dvc':      'Dividends - Common (payout)',
    'sstk':     'Sale of Common Stock (issuance)',
    'prstkc':   'Purchase of Common Stock (buybacks)',
    'emp':      'Number of Employees',
    'ppent':    'Net PP&E (tangibility)',
    'rect':     'Receivables - Total',
}

# Filter to variables that actually exist in our data
available_factors = [v for v in candidate_factors.keys() if v in df.columns]
print(f"Selected {len(available_factors)} candidate accounting variables:")
for i, var in enumerate(available_factors, 1):
    pct_valid = 100 * df[var].notna().sum() / len(df)
    print(f"  {i:2d}. {var:10s} - {candidate_factors[var]} ({pct_valid:.1f}% non-missing)")

In [ ]:
# ------------------------------------------------------------------
# Normalize each candidate factor cross-sectionally (within each month)
# Scale by total assets to make variables comparable across firms
# Then rank-normalize to ensure uniform distribution
# ------------------------------------------------------------------

# Create normalized versions: scale by total assets, then cross-sectional z-score
for var in available_factors:
    col_name = var + '_norm'
    
    # Scale by total assets (except 'at' itself and 'emp')
    if var not in ['at', 'emp']:
        raw = df[var] / df['at'].replace(0, np.nan)
    else:
        raw = df[var].copy()
    
    # Cross-sectional z-score normalization (within each month)
    df[col_name] = raw.groupby(df['MthCalDt']).transform(
        lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0
    )
    
    # Winsorize at 1st/99th percentiles
    lower = df[col_name].quantile(0.01)
    upper = df[col_name].quantile(0.99)
    df[col_name] = df[col_name].clip(lower=lower, upper=upper)

# Recompute NextRet since df may have changed
norm_cols = [v + '_norm' for v in available_factors]

print(f"Normalized {len(norm_cols)} variables cross-sectionally.")
print("\nSample statistics of normalized variables:")
print(df[norm_cols].describe().T[['mean', 'std', 'min', 'max']].to_string(float_format='{:.3f}'.format))

In [ ]:
# ------------------------------------------------------------------
# (b) Split into training and test periods
# Run single-factor regressions for each candidate
# ------------------------------------------------------------------

# Ensure NextRet is defined
if 'NextRet' not in df.columns:
    df['NextRet'] = df.groupby('PERMNO')['MthRet'].shift(-1)

# Use same 70/30 split
all_months = sorted(df['MthCalDt'].unique())
split_point = all_months[int(len(all_months) * 0.7)]

farm_train = df[df['MthCalDt'] <= split_point].copy()
farm_test = df[df['MthCalDt'] > split_point].copy()

print(f"Training period: {farm_train['MthCalDt'].min().strftime('%Y-%m')} to {farm_train['MthCalDt'].max().strftime('%Y-%m')}")
print(f"Test period:     {farm_test['MthCalDt'].min().strftime('%Y-%m')} to {farm_test['MthCalDt'].max().strftime('%Y-%m')}")

# Run single-factor regressions
results_insample = []
results_oos = []

for var in available_factors:
    norm_var = var + '_norm'
    
    # --- In-sample regression ---
    train_sub = farm_train.dropna(subset=[norm_var, 'NextRet'])
    if len(train_sub) < 100:
        continue
    
    X_is = sm.add_constant(train_sub[norm_var])
    y_is = train_sub['NextRet']
    model_is = OLS(y_is, X_is).fit(
        cov_type='cluster',
        cov_kwds={'groups': train_sub[['PERMNO', 'MthCalDt']].values}
    )
    
    # --- Out-of-sample regression ---
    test_sub = farm_test.dropna(subset=[norm_var, 'NextRet'])
    if len(test_sub) < 100:
        results_insample.append({
            'Variable': var,
            'Description': candidate_factors[var],
            'IS_coef': model_is.params[norm_var],
            'IS_tstat': model_is.tvalues[norm_var],
            'IS_pval': model_is.pvalues[norm_var],
            'IS_significant': model_is.pvalues[norm_var] < 0.05,
            'OOS_coef': np.nan,
            'OOS_tstat': np.nan,
            'OOS_pval': np.nan,
            'OOS_significant': False,
        })
        continue
    
    X_oos = sm.add_constant(test_sub[norm_var])
    y_oos = test_sub['NextRet']
    model_oos = OLS(y_oos, X_oos).fit(
        cov_type='cluster',
        cov_kwds={'groups': test_sub[['PERMNO', 'MthCalDt']].values}
    )
    
    # Same sign and significant OOS?
    same_sign = np.sign(model_is.params[norm_var]) == np.sign(model_oos.params[norm_var])
    
    results_insample.append({
        'Variable': var,
        'Description': candidate_factors[var],
        'IS_coef': model_is.params[norm_var],
        'IS_tstat': model_is.tvalues[norm_var],
        'IS_pval': model_is.pvalues[norm_var],
        'IS_significant': model_is.pvalues[norm_var] < 0.05,
        'OOS_coef': model_oos.params[norm_var],
        'OOS_tstat': model_oos.tvalues[norm_var],
        'OOS_pval': model_oos.pvalues[norm_var],
        'OOS_significant': model_oos.pvalues[norm_var] < 0.05,
        'Same_sign': same_sign,
        'Survives_OOS': model_is.pvalues[norm_var] < 0.05 and model_oos.pvalues[norm_var] < 0.05 and same_sign,
    })

results_ff = pd.DataFrame(results_insample)
print("\nFactor farming results computed.")

In [ ]:
# ------------------------------------------------------------------
# (c) Report results
# ------------------------------------------------------------------

print("="*80)
print("QUESTION 9: Factor Farming Results")
print("="*80)

# Display full results table
display_cols = ['Variable', 'Description', 'IS_coef', 'IS_tstat', 'IS_significant', 
                'OOS_coef', 'OOS_tstat', 'OOS_significant']
if 'Survives_OOS' in results_ff.columns:
    display_cols.append('Survives_OOS')

print("\nDetailed Results:")
print(results_ff[display_cols].to_string(index=False, float_format='{:.4f}'.format))

# Summary statistics
n_total = len(results_ff)
n_sig_is = results_ff['IS_significant'].sum()
n_sig_oos = results_ff['OOS_significant'].sum() if 'OOS_significant' in results_ff.columns else 0
n_survives = results_ff['Survives_OOS'].sum() if 'Survives_OOS' in results_ff.columns else 0
survival_rate = n_survives / n_sig_is * 100 if n_sig_is > 0 else 0

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Total candidate factors tested:            {n_total}")
print(f"Significant in-sample (p < 0.05):          {n_sig_is} ({100*n_sig_is/n_total:.0f}%)")
print(f"Significant out-of-sample (p < 0.05):      {n_sig_oos}")
print(f"Survive OOS (sig + same sign):             {n_survives}")
print(f"Survival rate (OOS survivors / IS sig):     {survival_rate:.1f}%")

In [ ]:
# ------------------------------------------------------------------
# (d) Visualization and discussion
# ------------------------------------------------------------------

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Plot: In-sample vs Out-of-sample t-statistics
valid = results_ff.dropna(subset=['IS_tstat', 'OOS_tstat'])
colors = ['green' if s else 'red' for s in valid.get('Survives_OOS', [False]*len(valid))]
ax1.scatter(valid['IS_tstat'], valid['OOS_tstat'], c=colors, s=80, alpha=0.7, edgecolors='black')
ax1.axhline(y=1.96, color='gray', linestyle='--', alpha=0.5, label='t=1.96')
ax1.axhline(y=-1.96, color='gray', linestyle='--', alpha=0.5)
ax1.axvline(x=1.96, color='gray', linestyle='--', alpha=0.5)
ax1.axvline(x=-1.96, color='gray', linestyle='--', alpha=0.5)

# Label each point
for _, row in valid.iterrows():
    ax1.annotate(row['Variable'], (row['IS_tstat'], row['OOS_tstat']),
                fontsize=7, ha='left', va='bottom')

ax1.set_xlabel('In-Sample t-statistic', fontsize=11)
ax1.set_ylabel('Out-of-Sample t-statistic', fontsize=11)
ax1.set_title('Factor Farming: In-Sample vs OOS t-statistics', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot: Bar chart of survival
categories = ['Total\nCandidates', 'Significant\nIn-Sample', 'Significant\nOut-of-Sample', 'Survive\nOOS']
counts = [n_total, n_sig_is, n_sig_oos, n_survives]
bar_colors = ['steelblue', 'orange', 'coral', 'green']
ax2.bar(categories, counts, color=bar_colors, edgecolor='black', alpha=0.8)
for i, v in enumerate(counts):
    ax2.text(i, v + 0.3, str(v), ha='center', va='bottom', fontweight='bold', fontsize=12)
ax2.set_title('Factor Farming: Attrition Funnel', fontweight='bold')
ax2.set_ylabel('Number of Factors')

plt.tight_layout()
plt.savefig('factor_farming.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("(d) Discussion: Why Do Most In-Sample Factors Fail OOS?")
print("="*60)
print("""
Several reasons explain why most in-sample factors fail out-of-sample:

1. DATA MINING / OVERFITTING: With hundreds of variables to test, some will
   appear significant purely by chance (multiple testing problem). At a 5%
   significance level, we expect ~1 in 20 random variables to appear significant.

2. STRUCTURAL BREAKS: Market conditions, regulation, and investor behavior
   change over time. A factor that worked in the past may no longer work because
   the underlying economic mechanism has changed.

3. ARBITRAGE AND CROWDING: Once a factor is discovered and widely traded,
   the premium may shrink or disappear as capital flows in to exploit it.
   This is especially true for factors published in academic literature.

4. TRANSACTION COSTS: Even if a factor has statistical significance, the
   premium may be too small to survive after trading costs, especially for
   strategies involving small or illiquid stocks.

5. SAMPLE-SPECIFIC PATTERNS: Some factors may capture patterns unique to
   the training period (e.g., a bubble, a crisis) that don't repeat.

This is a well-documented problem in finance. Harvey, Liu, and Zhu (2016)
estimate that the t-statistic threshold should be raised to ~3.0 to account
for the hundreds of factors that have been tested in the literature.
""")